In [1]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import numpy as np
import spacy

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
import gc


import matplotlib.pyplot as plt
import glob
import plotly.express as px
import warnings
import plotly as py
import plotly.graph_objs as go

import string
import re
nlp = spacy.load('en_core_web_lg')
import os

In [ ]:

DATA_PATH = "/media/kibria/624309e6-cae1-4eca-a603-4ebc61fc0d9b/data/archive (2)/covid-19_vaccine_tweets_with_sentiment.csv"

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Dataset not found at {DATA_PATH}. Please download and place it in this folder.")


try:
    df = pd.read_csv(DATA_PATH)
except Exception:
    df = pd.read_csv(DATA_PATH, encoding='latin-1')

print('Raw data shape:', df.shape)
print('Columns:', list(df.columns))

Raw data shape: (6000, 3)
Columns: ['tweet_id', 'label', 'tweet_text']


In [12]:
df.head()

,tweet_id,label,tweet_text
0,1.360342e+18,1,"4,000 a day dying from the so called Covid-19 ..."
1,1.382896e+18,2,Pranam message for today manifested in Dhyan b...
2,1.375673e+18,2,Hyderabad-based ?@BharatBiotech? has sought fu...
3,1.381311e+18,1,"Confirmation that Chinese #vaccines ""dont hav..."
4,1.362166e+18,3,"Lab studies suggest #Pfizer, #Moderna vaccines..."


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tweet_id    6000 non-null   float64
 1   label       6000 non-null   int64  
 2   tweet_text  6000 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 140.8+ KB


In [16]:
labels=df.groupby("label").agg({'tweet_text':'count'}).rename(columns={'tweet_text':'tweet_count'}).sort_values(by="tweet_count", ascending=False)

In [17]:
fig1 = px.bar(labels, x="tweet_count", y=labels.index.astype('str'), orientation='h', hover_data=['tweet_count'],labels={'y':'Labels'}, title='Label Counts')
fig1.update_layout(yaxis={'categoryorder':'total ascending'})
fig1.show()

In [23]:
def removeStopwords(text):
    doc = nlp(text)
    clean_text = ' '
    for txt in doc:
        if (txt.is_stop == False):
            clean_text = clean_text + " " + str(txt)        
    return clean_text

def removePunctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation)).replace(" s ", " ")

def removeLinks(text):
    clean_text = re.sub('https?://\S+|www\.\S+', '', text)
    return clean_text

def removeNumbers(text):
    clean_text = re.sub(r'\d+', '', text)
    return clean_text
def removenewline(text):
    clean_text = re.sub(r'(\n+)', '', text)
    return clean_text


import emoji
#does the text contain an emoji?
def text_has_emoji(text):
    for character in text:
        if character in emoji.EMOJI_DATA:
            return True
    return False
#remove the emoji
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')


In [24]:
def clean(text):
    if text_has_emoji(text):
        text=deEmojify(text)
    text = text.lower()
    text =removenewline(text)
    text = removeStopwords(text)
    text = removePunctuations(text)
    text = removeNumbers(text)
    text = removeLinks(text)
    return text

In [ ]:
df['text_clean'] = df['tweet_text'].apply(clean)
df['label'] = df['label'].map({1:0, 2:1, 3:2})
df.drop("tweet_text",axis=1).to_csv('mycsvfile.csv',index=False)

In [ ]:
# Set seeds for reproducibility
import random
import numpy as np
from collections import Counter
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)


nlp = spacy.load("en_core_web_sm")

# Sample dataset (replace with your data)
texts = [
    "This is a sample tweet about health.",
    "Another example with positive sentiment!",
    "Bad news about vaccine side effects."
]
labels = [0, 1, 2]  #  0=neutral, 1=positive, 2=negative

# Tokenize texts using SpaCy
def tokenize(text):
    return [token.text.lower() for token in nlp(text) if not token.is_punct and not token.is_space]

tokenized_texts = [tokenize(text) for text in texts]

# Build vocabulary 
counter = Counter()
for tokens in tokenized_texts:
    counter.update(tokens)

# Keep words that occur at least once (can set min_freq>1)
vocab = {word: idx + 2 for idx, (word, freq) in enumerate(counter.items())}  
vocab["<pad>"] = 0
vocab["<unk>"] = 1

# Function to numericalize tokenized text
def numericalize(tokens, vocab):
    return [vocab.get(token, vocab["<unk>"]) for token in tokens]

numericalized_texts = [numericalize(tokens, vocab) for tokens in tokenized_texts]

# pad sequences to max length
def pad_sequence(seq, max_len, pad_value=0):
    return seq + [pad_value] * (max_len - len(seq))

max_len = max(len(seq) for seq in numericalized_texts)
padded_texts = [pad_sequence(seq, max_len) for seq in numericalized_texts]

# Convert to tensors
input_tensor = torch.tensor(padded_texts, dtype=torch.long)
label_tensor = torch.tensor(labels, dtype=torch.long)

print("Vocabulary:", vocab)
print("Padded input tensor shape:", input_tensor.shape)
print("Labels tensor:", label_tensor)

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.